# MC Code RAG Workshop


### Dependencies


In [ ]:
# %pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai chromadb bs4 python-dotenv

In [1]:
import os
from dotenv import load_dotenv,find_dotenv

# Use this line of code if you have a local .env file
load_dotenv(find_dotenv()) 

# LangSmith
import getpass
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.environ["LANGCHAIN_API_KEY"] #getpass.getpass()
 
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader, TextLoader, PyPDFLoader, PyPDFium2Loader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document

# Txt loader
# loader = TextLoader("./docs/WMX3UserManual_a.txt")
loader = TextLoader("./docs/WMX3API_MCEval_Samplecodes.txt")
docs = loader.load()
# docs[0].page_content[:100000]
len(docs)

# # Text chunk 
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)
# splits = text_splitter.split_documents(docs)

#Sample code chunk with dedicated separators
separators = ['#.']  # Adjust based on actual document structure
# separators = ['}']  # Adjust based on actual document structure
text_splitter = RecursiveCharacterTextSplitter(separators=separators, keep_separator=True, chunk_size=1000, chunk_overlap=200, add_start_index=True)
splits = text_splitter.split_documents(docs)


True

In [ ]:


embedding_model=OpenAIEmbeddings(model="text-embedding-3-large")   #text-embedding-3-large   #text-embedding-ada-002    #text-embedding-3-small

# If txt vectorstore exists
# if os.path.exists("Vectorstore/chromadb"):
        # vectorstore = Chroma(
                #     embedding_function=embedding_model,
                #     persist_directory="Vectorstore/chromadb",
                #     ) 
# else:
        # Load from chunks and save to disk
        # vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model, persist_directory="Vectorstore/chromadb") 

# If pdf vectorstore exists
vectorstore_path = "Vectorstore/chromadb-MCCoder"
 # Load from chunks and save to disk
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model,  persist_directory=vectorstore_path) 

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})

retrieved_docs = retriever.invoke("Supported PC Hardware ")